In [1]:
from sql_blocks import *

In [2]:
Select.join_type = JoinType.LEFT
OrderBy.sort = SortType.DESC

In [3]:
baixo_risco=SubSelect( 'Risco', cpf_cliente=[GroupBy, Field], nota=Having.avg(Where.lt(5)) )

In [4]:
melhores_clientes = Select('Cliente c', nome=Field, cpf=[PrimaryKey, baixo_risco])

In [5]:
parcelas_em_aberto = Select(
    'Parcela', valor=[Field, OrderBy],
    cpf_cliente=melhores_clientes,  # --- Bons pagadores ...
    situacao=Where.eq('em aberto')  # --- ...com parcelas para negociar
)

In [6]:
print(parcelas_em_aberto)

SELECT
	par.valor,
	nome
FROM
	Parcela par
	LEFT JOIN Cliente cli ON (par.cpf_cliente = cli.cpf)
WHERE
	cli.cpf IN (SELECT ris.cpf_cliente FROM Risco ris GROUP BY ris.cpf_cliente HAVING Avg(ris.nota) < 5)
	AND par.situacao = 'em aberto'
ORDER BY
	par.valor DESC


In [7]:
produto = Select(
    Produto=Table('nome,valor'),
    classe_prod=Case('valor').when(
        Where.gte(3000), 'clase A',
    ).when(
        Where.gte(1000), 'classe B'
    ).else_value('classe C'),
    restricao=Where.is_null(),
    categoria=Not.list(['Higiene', 'Limpeza', 'Automotivos'])
)

In [8]:
print(produto)

SELECT
	pro.nome,
	pro.valor,
	CASE 
		WHEN pro.valor >= 3000 THEN 'clase A'
		WHEN pro.valor >= 1000 THEN 'classe B'
		ELSE 'classe C'
	END AS classe_prod
FROM
	Produto pro
WHERE
	pro.restricao IS NULL
	AND NOT pro.categoria IN ('Higiene','Limpeza','Automotivos')
